In [11]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding 
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import keywrap
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import hashlib
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.primitives import serialization, hashes
import os




In [12]:
#1,2.Step-by-Step Procedures
# read txt/csv
file_path = "C:/Users/Martin Lin/Desktop/SE6003/Ass/SE6003A1/SE6003A1/plaintext_file.txt"  # input the file path here
digest_file_path = "C:/Users/Martin Lin/Desktop/SE6003/Ass/SE6003A1/SE6003A1/digest_file.hex"# output the file path here
with open(file_path, 'rb') as file:
    file_data = file.read()  # 读取整个文件为字节数据
    # file_data=file_data.decode('utf-8')
print(file_data)

# Initialize SHA-512 hash object
sha512_hash = hashlib.sha512()
sha512_hash.update(file_data)
digest_hex = sha512_hash.hexdigest()

print(
    "--------------------------------------------------------------------"
)
# Print the SHA-512 hash
print("SHA-512 Hash:", digest_hex)

# output the digest_file
with open(digest_file_path, 'w') as digest_file:
    digest_file.write(digest_hex)  
    


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [13]:
#3,4.RSA-4096 key pair,Sign plaintext .file 
plaintext_file_path = "C:/Users/Martin Lin/Desktop/SE6003/Ass/SE6003A1/SE6003A1/plaintext_file.txt" 

#generate the private key
private_key = rsa.generate_private_key(public_exponent=65537,key_size=4096,backend=default_backend())

# Save the private key in PEM format
with open("RSA_private_key.pem", "wb") as private_key_file:
    private_key_file.write(
        private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        )
    )

# Save the public key in PEM format
public_key = private_key.public_key()
with open("RSA_public_key.pem", "wb") as public_key_file:
    public_key_file.write(
        public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
    )
print("RSA-4096 key pair generated and saved.")



#Sign the plaintext_file using the private key
signed_file_path = "signed_file"
#read the plaintext first
with open(plaintext_file_path, 'rb') as file:
    plaintext_data = file.read()

# Create the signature by the function using the private key
signature = private_key.sign(
    plaintext_data,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA512()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA512()
)

# Save the signature to a file
with open(signed_file_path, 'wb') as signed_file:
    signed_file.write(signature)

print(f"File signed and signature saved to {signed_file_path}.")

# Step 3: Verify the signature using the public key
try:
    public_key.verify(
        signature,
        plaintext_data,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA512()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA512()
    )
    print("Signature is valid.")
except InvalidSignature:
    print("Signature is invalid.")

RSA-4096 key pair generated and saved.
File signed and signature saved to signed_file.
Signature is valid.


In [14]:
#verify the RSA-4096 key pair

from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.exceptions import InvalidSignature

# Paths to the public key, plaintext file, and the signature file
public_key_path = "C:/Users/Martin Lin/Desktop/SE6003/Ass/SE6003A1/SE6003A1/RSA_public_key.pem"
plaintext_file_path = "C:/Users/Martin Lin/Desktop/SE6003/Ass/SE6003A1/SE6003A1/plaintext_file.txt"
signature_file_path = "C:/Users/Martin Lin/Desktop/SE6003/Ass/SE6003A1/SE6003A1/signed_file"

def verify_signature(public_key_path, plaintext_file_path, signature_file_path):
    # Load the public key from a PEM file
    with open(public_key_path, "rb") as public_key_file:
        public_key = serialization.load_pem_public_key(public_key_file.read())
    
    # Read the plaintext data from the file
    with open(plaintext_file_path, "rb") as plaintext_file:
        plaintext_data = plaintext_file.read()

    # Read the binary signature from the file
    with open(signature_file_path, "rb") as signature_file:
        signature = signature_file.read()  # Directly read binary signature

    # Verify the signature
    try:
        public_key.verify(
            signature,
            plaintext_data,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA512()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA512()
        )
        print("Signature is valid.")
        return True
    except InvalidSignature:
        print("Signature is invalid.")
        return False



# Call the verification function
is_valid = verify_signature(public_key_path, plaintext_file_path, signature_file_path)

if is_valid:
    print("The signature matches and is valid.")
else:
    print("The signature is invalid.")


Signature is invalid.
The signature is invalid.


In [15]:
# Check aes_key and decrypted_AES_key
with open("AES_key.txt") as aes_file:
    aes_key = aes_file.read()
print(aes_key)
with open("decrypted_AES_key.txt") as decrypted_aes_key_file:
    decrypted_key = decrypted_aes_key_file.read()
print(decrypted_key)
if aes_key  == decrypted_key:
    print("Same!!!!!!!!")
else:
    print("错啦")

# print("比较一下")
# decrypted_aes_key_binary_string = ''.join(format(byte, '08b') for byte in decrypted_aes_key)
# if aes_key_binary_string == decrypted_aes_key_string:
#     print(True)
# else:
#     print(False)
#     print(type(aes_key))
#     print(type(decrypted_aes_key_string))
# print(aes_key_binary_string)
# print(aes_key)

0000111011010111111100111101011101111011110100101010100111011110110010000100010110110110000010011000111001011101100100111101001111001100010101111111101011000101010001101000110010010101001111100101111110000010010111110010011110000100101100000011111001001001
0000111011010111111100111101011101111011110100101010100111011110110010000100010110110110000010011000111001011101100100111101001111001100010101111111101011000101010001101000110010010101001111100101111110000010010111110010011110000100101100000011111001001001
Same!!!!!!!!


In [16]:
#3-4generate,Encrypt,Decrypt AES

# Step 5: Generate an AES-256 key (32 bytes == 256 bits)
aes_key = os.urandom(32)  


# Convert each byte to  binary string 
print("aes_key:")
print(aes_key)
aes_key_binary_string = ''.join(format(byte, '08b') for byte in aes_key)
print(aes_key_binary_string)
# Save AES key to a file (binary format)
with open("AES_key.txt", "w") as aes_file:
    aes_file.write(aes_key_binary_string )

with open("AES_key_original.txt", "wb") as original_aes_key_file:
    original_aes_key_file.write(aes_key)

print("------------------------------------------------------------------------")

# Step 6: Encrypt the AES key using the RSA public key and save it as encrypted_AES_key.txt
print("Encrypt_key:")
# Load RSA public key from PEM file (assuming you already generated and saved this key)
with open("RSA_public_key.pem", "rb") as key_file:
    public_key = serialization.load_pem_public_key(key_file.read(), backend=default_backend())

# Encrypt the AES key using RSA public key
encrypted_aes_key = public_key.encrypt(
    aes_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# Save the encrypted AES key to a file
# encrypted_aes_key_binary_string = ''.join(format(byte, '08b') for byte in encrypted_aes_key)

with open("encrypted_AES_key.txt", "wb") as binary_file:
    binary_file.write(encrypted_aes_key)

print(encrypted_aes_key)
print("------------------------------------------------------------------------")

# Step 7: Decrypt encrypted_AES_key.txt using the RSA private key and save as decrypted_AES_key.txt

# Load RSA private key from PEM file (assuming you already generated and saved this key)
with open("RSA_private_key.pem", "rb") as key_file:
    private_key = serialization.load_pem_private_key(key_file.read(), password=None, backend=default_backend())

# Read the encrypted AES key from the file
with open("encrypted_AES_key.txt", "rb") as encrypted_file:
    encrypted_aes_key = encrypted_file.read()

# Decrypt the AES key using the RSA private key
decrypted_aes_key = private_key.decrypt(
    encrypted_aes_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
print("decrypted_aes_key: ")
print(decrypted_aes_key)
# with open("decrypted_AES_key_original.txt", "wb") as decrypted_file_original:
#     decrypted_aes_key_original = decrypted_file_original.write(decrypted_aes_key)
decrypted_aes_key_string = ''.join(format(byte, '08b') for byte in decrypted_aes_key)
print(decrypted_aes_key_string)
print("------------------------------------------------------------------------")
# # Save the decrypted AES key to a file
with open("decrypted_AES_key.txt", "w") as decrypted_file:
    decrypted_file.write(decrypted_aes_key_string)

# print("AES key decrypted and saved as decrypted_AES_key.txt.")

# Check Check Check
with open("AES_key.txt") as aes_file:
    aes_key = aes_file.read()
with open("decrypted_AES_key.txt") as decrypted_aes_key_file:
    decrypted_key = decrypted_aes_key_file.read()
if aes_key  == decrypted_key:
    print("Same!!!!!!!!")
else:
    print("错啦")

# print("比较一下")
# decrypted_aes_key_binary_string = ''.join(format(byte, '08b') for byte in decrypted_aes_key)
# if aes_key_binary_string == decrypted_aes_key_string:
#     print(True)
# else:
#     print(False)
#     print(type(aes_key))
#     print(type(decrypted_aes_key_string))
# print(aes_key_binary_string)
# print(aes_key)

aes_key:
b'\x95\xb8~\x84\xa8\x14\x1d\x1bznX^4G\xa7\x95\x82v\xb6?\x1fCr&\xe1dY\xba\x92\xc21('
1001010110111000011111101000010010101000000101000001110100011011011110100110111001011000010111100011010001000111101001111001010110000010011101101011011000111111000111110100001101110010001001101110000101100100010110011011101010010010110000100011000100101000
------------------------------------------------------------------------
Encrypt_key:
b'\x11%\xc72(\xbe!\xc8\xc2\xc2\xb2\xb5h\xbc\x05I\x9e\xa9\xc6\x0f\xb6d\x1f]_\xe6\xed\x82\x7f\xca8/\xa6\xbf/e\xb2A\x8c\xc6\x1c\x13@\x07R\x17K\xd9{\ne\x8fl\x9f\xf4&\x04()s\xe6\x87\xbe\xb4\x99l\x93\xbf\xdb0\xabss\n\tS\xac\xbdCZ\xcb\x9b\xab\xba\xc1\x0c\xe8\xb9\x90\xff\x9a\x04|\x013\xb1"\xc82\xd2-\'\xac\xbb\xdb\xa7\xf44}qw\xe4w\xb5H\x9f\x8e4"\x8f\xb7\x08\xde\xfb#\x1b\xb6+\xc7\xf3\x99Ioo\xe7\xbb\xc58E\xcc\xe0\xde\x82\x98;\xd0a?|0S\xbb\xc4\xcc>e\x17\xa0\x04\x87\x96\x8c\x916v\xca\x9e\x97\x9c\xd0\xa9,\xc4\xa0\x16\x146\x00\x95\xf9\x08ig2\xc0\xfc\xe5\xc6\xe4\x14t\xa9\xd

In [17]:
# 8. Encrypt plaintext file by using the AES key in 
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os
# Load AES key
with open("AES_key.txt", "rb") as aes_file:
    aes_key = aes_file.read()

# Convert binary string to bytes
aes_key = int(aes_key.decode(), 2).to_bytes(len(aes_key) // 8, byteorder='big')

# Load plaintext data
with open("plaintext_file.txt", 'rb') as file:
    plaintext_data = file.read()

# Apply PKCS7 padding to the plaintext (AES block size is 128 bits = 16 bytes)
padder = padding.PKCS7(128).padder()
padded_data = padder.update(plaintext_data) + padder.finalize()

# Create AES cipher in ECB mode 
cipher = Cipher(
    algorithms.AES(aes_key),
    modes.ECB(),
    backend=default_backend()
)
encryptor = cipher.encryptor()

# Encrypt the padded plaintext
ciphertext = encryptor.update(padded_data) + encryptor.finalize()

# Save the ciphertext
with open("ciphertext_file", 'wb') as ciphertext_file:
    ciphertext_file.write(ciphertext)

print("File encrypted using AES-ECB.")


File encrypted using AES-ECB.


In [18]:
#verify the ECB en/decryption

with open("ciphertext_file", 'rb') as ciphertext_file:
    ciphertext = ciphertext_file.read()

# Step 6: Create the same AES cipher in ECB mode for decryption
decryptor = cipher.decryptor()

# Step 7: Decrypt the ciphertext
decrypted_padded_data = decryptor.update(ciphertext) + decryptor.finalize()

# Remove PKCS7 padding from the decrypted data
unpadder = padding.PKCS7(128).unpadder()
decrypted_data = unpadder.update(decrypted_padded_data) + unpadder.finalize()

# Save the decrypted data to a file 
with open("decrypted_file", 'wb') as decrypted_file:
    decrypted_file.write(decrypted_data)

print("File decrypted and saved as 'decrypted_file'.")



File decrypted and saved as 'decrypted_file'.


In [19]:
# --- Verification Part ---
with open("plaintext_file.txt", 'r') as plaintext_file:
    plaintext_data = plaintext_file.read()

with open("decrypted_file", 'r') as decrypted_file:
    decrypted_data = decrypted_file.read()
    
if plaintext_data == decrypted_data:
    print("The decrypted file is identical to the original plaintext.")
else:
    print("不一样啊！！！出事啦啦啦啦啦啦啦！！！！！.")


UnicodeDecodeError: 'gbk' codec can't decode byte 0xdf in position 37: illegal multibyte sequence

In [70]:
#step.11
# Load the decrypted file
with open("decrypted_file", 'rb') as decrypted_file:
    decrypted_data = decrypted_file.read()

# Compute the SHA-512 digest of the decrypted data
sha512_hash = hashlib.sha512(decrypted_data).digest()

# Convert the digest to a hexadecimal format
hex_digest = sha512_hash.hex()

# Save the hexadecimal digest to a file
with open("digest_decrypted_file.hex", 'w') as hex_file:
    hex_file.write(hex_digest)

print("SHA-512 digest of the decrypted file has been saved to 'digest_decrypted_file.hex'.")

SHA-512 digest of the decrypted file has been saved to 'digest_decrypted_file.hex'.


In [71]:
#verify digest_decrypted_file.hex
with open("plaintext_file.txt", 'rb') as plaintext:
    plaintext_data = plaintext.read()
# Load the decrypted file
with open("decrypted_file", 'rb') as decrypted_file:
    decrypted_data = decrypted_file.read()

# Compute the SHA-512 hash
computed_sha512_hash = hashlib.sha512(plaintext_data).hexdigest()
decrypted_data_sha512_hash = hashlib.sha512(decrypted_data).hexdigest()
# Load the saved digest from the file
with open("digest_decrypted_file.hex", 'r') as hex_file:
    saved_sha512_hash = hex_file.read()

with open("digest_file.hex", 'r') as digest_file:
    saved_digest_file_data= digest_file.read()
    
# Verify if they match
if computed_sha512_hash == saved_sha512_hash == decrypted_data_sha512_hash == saved_digest_file_data:
    print("The hash matches! Verification successful.")
else:
    print("出错了，不一样")



The hash matches! Verification successful.
